<a href="https://colab.research.google.com/github/gd-Sahat/TL-LEO-POD/blob/main/starlink_eph_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def parse_modified_itc(content):
    """
    Parse Modified ITC ephemeris file format
    Args:
        content (str): File content as a string
    Returns:
        pd.DataFrame: DataFrame with ephemeris data
    """
    lines = content.strip().split('\n')

    # Validate minimum file structure
    if len(lines) < 5:
        raise ValueError("Incomplete file: Insufficient lines")

    # Skip header lines (first 3) and UVW marker (4th line)
    data_lines = lines[4:]
    if not data_lines:
        raise ValueError("No ephemeris data found after header")

    data = []
    block_size = 4
    num_blocks = len(data_lines) // block_size

    for i in range(num_blocks):
        start_idx = i * block_size
        block = data_lines[start_idx:start_idx + block_size]

        # Parse timestamp line
        ts_line = block[0].split()
        if len(ts_line) < 7:
            continue

        # Parse timestamp (yyyyDOYhhmmss.sss)
        ts_str = ts_line[0]
        try:
            year = int(ts_str[:4])
            doy = int(ts_str[4:7])
            hour = int(ts_str[7:9])
            minute = int(ts_str[9:11])
            second = int(ts_str[11:13])
            millisec = int(ts_str[14:17])  # Skip decimal at index 13

            # Create base datetime (Jan 1 + DOY offset)
            base_date = datetime(year, 1, 1) + timedelta(days=doy-1)
            timestamp = base_date.replace(
                hour=hour, minute=minute, second=second,
                microsecond=millisec*1000
            )
        except Exception as e:
            raise ValueError(f"Timestamp parsing error: {ts_str}") from e

        # Parse position and velocity (km and km/s)
        try:
            x, y, z = map(float, ts_line[1:4])
            vx, vy, vz = map(float, ts_line[4:7])
        except ValueError as e:
            raise ValueError(f"Position/velocity parsing error: {ts_line}") from e

        # Parse covariance matrix (6x6 lower triangular in UVW frame)
        cov_values = []
        for cov_line in block[1:4]:
            try:
                cov_values.extend(map(float, cov_line.split()))
            except ValueError as e:
                raise ValueError(f"Covariance parsing error: {cov_line}") from e

        # Ensure we have exactly 21 covariance values
        if len(cov_values) != 21:
            raise ValueError(f"Invalid covariance count: {len(cov_values)}/21")

        # Create full record
        record = [timestamp, x, y, z, vx, vy, vz] + cov_values
        data.append(record)

    # Create column names
    columns = ['timestamp', 'x', 'y', 'z', 'vx', 'vy', 'vz']
    cov_columns = []
    idx = 0
    for i in range(1, 7):  # For 6x6 matrix
        for j in range(1, i+1):
            idx += 1
            cov_columns.append(f'cov_{i}_{j}')

    return pd.DataFrame(data, columns=columns + cov_columns)

# Example usage
if __name__ == "__main__":
    with open('MEME_63365_STARLINK-33741_1511732_Operational_1433007180_UNCLASSIFIED.txt', 'r') as f:
        content = f.read()

    df = parse_modified_itc(content)
    print(f"Parsed {len(df)} ephemeris records")
    print(df.head())

    # Export to CSV
    df.to_csv('starlink_ephemerides.csv', index=False)
    print("CSV saved to starlink_ephemerides.csv")

Parsed 4321 ephemeris records
            timestamp            x            y            z        vx  \
0 2025-05-31 17:32:42  4506.314549 -2039.340656  4693.545455 -0.068742   
1 2025-05-31 17:33:42  4492.019131 -1615.483082  4868.539232 -0.407586   
2 2025-05-31 17:34:42  4457.441760 -1184.331664  5021.488706 -0.744555   
3 2025-05-31 17:35:42  4402.740463  -747.833271  5151.703791 -1.078131   
4 2025-05-31 17:36:42  4328.163987  -307.959008  5258.597024 -1.406812   

         vy        vz       cov_1_1       cov_2_1       cov_2_2  ...  \
0  6.992817  3.096010  4.809085e-07 -3.940340e-07  7.887019e-07  ...   
1  7.130443  2.734918  5.307962e-07 -4.617230e-07  9.002565e-07  ...   
2  7.235863  2.361481  5.836655e-07 -5.373340e-07  1.030340e-06  ...   
3  7.308606  1.977389  6.393212e-07 -6.211340e-07  1.181045e-06  ...   
4  7.348350  1.584379  6.975291e-07 -7.133230e-07  1.354524e-06  ...   

        cov_5_2       cov_5_3       cov_5_4       cov_5_5       cov_6_1  \
0  4.161049e-10 -

In [ ]:
df

,timestamp,x,y,z,vx,vy,vz,cov_1_1,cov_2_1,cov_2_2,...,cov_5_2,cov_5_3,cov_5_4,cov_5_5,cov_6_1,cov_6_2,cov_6_3,cov_6_4,cov_6_5,cov_6_6
0,2025-05-31 17:32:42,4506.314549,-2039.340656,4693.545455,-0.068742,6.992817,3.096010,4.809085e-07,-3.940340e-07,7.887019e-07,...,4.161049e-10,-1.617466e-12,-8.412099e-13,5.121449e-13,-2.093561e-13,3.181256e-13,1.675469e-09,3.278682e-16,-1.885898e-15,5.413313e-12
1,2025-05-31 17:33:42,4492.019131,-1615.483082,4868.539232,-0.407586,7.130443,2.734918,5.307962e-07,-4.617230e-07,9.002565e-07,...,4.860827e-10,-1.990052e-12,-9.452924e-13,5.640888e-13,1.186906e-14,-9.080885e-14,1.891002e-09,1.024923e-15,-2.476141e-15,5.173196e-12
2,2025-05-31 17:34:42,4457.441760,-1184.331664,5021.488706,-0.744555,7.235863,2.361481,5.836655e-07,-5.373340e-07,1.030340e-06,...,5.642829e-10,-2.426030e-12,-1.057322e-12,6.193962e-13,2.664044e-13,-5.849492e-13,2.074136e-09,1.826749e-15,-3.062725e-15,4.903026e-12
3,2025-05-31 17:35:42,4402.740463,-747.833271,5151.703791,-1.078131,7.308606,1.977389,6.393212e-07,-6.211340e-07,1.181045e-06,...,6.510020e-10,-2.921010e-12,-1.177230e-12,6.778902e-13,5.520684e-13,-1.159492e-12,2.221572e-09,2.718496e-15,-3.632575e-15,4.607940e-12
4,2025-05-31 17:36:42,4328.163987,-307.959008,5258.597024,-1.406812,7.348350,1.584379,6.975291e-07,-7.133230e-07,1.354524e-06,...,7.464488e-10,-3.469064e-12,-1.304871e-12,7.393306e-13,8.668593e-13,-1.807295e-12,2.330652e-09,3.682175e-15,-4.172029e-15,4.293528e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4316,2025-06-03 17:28:42,3116.591456,3902.532568,4642.886501,-3.054373,6.243064,-3.190835,1.600000e-01,-1.975151e-17,3.600000e+01,...,-8.407918e-20,-1.790407e-05,5.464931e-10,1.119642e-08,4.883743e-06,1.790407e-03,-3.441805e-20,-7.829613e-08,2.135705e-10,9.919235e-08
4317,2025-06-03 17:29:42,2926.476450,4268.087035,4441.146768,-3.281898,5.935275,-3.533841,1.600000e-01,-7.745274e-15,3.600000e+01,...,-7.983118e-19,-1.712469e-05,5.788423e-10,1.112054e-08,5.408262e-06,1.712468e-03,2.278725e-19,-7.488355e-08,2.364948e-10,9.164248e-08
4318,2025-06-03 17:30:42,2723.145704,4614.367537,4219.294286,-3.494626,5.600648,-3.860880,1.600000e-01,4.406789e-15,3.600000e+01,...,4.324563e-19,-1.626776e-05,6.007123e-10,1.104102e-08,5.908247e-06,1.626776e-03,-1.318665e-19,-7.113215e-08,2.583430e-10,8.372932e-08
4319,2025-06-03 17:31:42,2507.516093,4939.807955,3978.331675,-3.691594,5.240689,-4.170467,1.600000e-01,-4.051227e-15,3.600000e+01,...,-3.075583e-19,-1.533721e-05,6.117090e-10,1.095929e-08,6.381436e-06,1.533721e-03,-1.329768e-19,-6.705916e-08,2.790167e-10,7.559621e-08
